# MEIRL

In [3]:
import pandas as pd
import numpy as np
import random

In [4]:
area = "oxford"
filename = "data/" + area + "/adjacency_matrix.csv"
adjacency_matrix = pd.read_csv(filename, index_col=0)

In [5]:
expert_feature_count = adjacency_matrix.copy()
expert_feature_count["value"] = 0
expert_feature_count = expert_feature_count["value"]
expert_feature_count = expert_feature_count.to_frame()
learner_feature_count = expert_feature_count.copy()
test_feature_count = expert_feature_count.copy()

In [6]:
# cross_validation
all_data = 400
training_number = int(all_data * 0.75)
test_number = int(all_data * 0.25)

# Import expert trajectory

### prepare dataset for training

In [7]:
for i in range (1,training_number):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(expert_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        expert_feature_count.loc[int(j), ["value"]] = new_count
expert_feature_count = (expert_feature_count - expert_feature_count.min()) / expert_feature_count.max() - expert_feature_count.min()    

### prepare for testing

In [8]:
for i in range (training_number + 1 , all_data):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(test_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        test_feature_count.loc[int(j), ["value"]] = new_count
test_feature_count = (test_feature_count - test_feature_count.min()) / test_feature_count.max() - test_feature_count.min()    

# Let's train

In [16]:
#Initialization of reward parameter
reward_parameter = expert_feature_count.copy()
reward_parameter.columns = ["value"]
reward_parameter["value"] = np.random.random_sample(reward_parameter.shape[0])

In [17]:
def update_position(current_position, eps, reward):
    action_list = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
    value_list = reward[reward.index.isin(action_list)]

    if np.random.random() > eps:
        updated_position = random.choice(value_list[value_list["value"] == max(value_list["value"])].index.tolist())
#         gained_reward = value_list.loc[updated_position, "value"]
    else:
        updated_position = random.choice(value_list["value"].index.tolist())
#         gained_reward = value_list.loc[updated_position, "value"]

    return updated_position

In [18]:
# how many iterations?
Y = []
iteration = 50
for i in range (iteration):
    epsilon = 1 / (i+1)  
    learner_feature_count["value"] = 0
    for j in range (1,training_number):
        start_position = random.choice(adjacency_matrix.columns)
        current_position = start_position
        # agent will walk for 50 steps
        for k in range(50):
            current_position = str(update_position(current_position, epsilon, reward_parameter))
            learner_feature_count.loc[int(current_position), ["value"]] += 1
    
    learner_feature_count = (learner_feature_count - learner_feature_count.min()) / (learner_feature_count.max() - learner_feature_count.min() )   
    gradient = (expert_feature_count - learner_feature_count) 
    reward_parameter = (gradient*0.2) + reward_parameter
    reward_parameter = (reward_parameter - reward_parameter.min()) / (reward_parameter.max() - reward_parameter.min() )   
    
    difference = gradient.abs().sum().values[0]
    Y.append(difference)
    print(i, difference)
    
    correlation_dataframe = pd.concat([learner_feature_count, test_feature_count], axis=1)
    correlation = correlation_dataframe.corr().values[0][1]
    print(correlation)

0 42.7354693499
0.23963837345
1 41.6497620027
0.135595961085
2 41.109169519
0.340379056261
3 42.2172480763
0.354762092823
4 43.4519575132
0.25385928883
5 40.4697115385
0.40446037776
6 40.0842689111
0.40375322035
7 41.6175323048
0.482494695202
8 41.9120879121
0.421552139816
9 39.901423167
0.380701881758
10 45.1938212038
0.362219734995
11 44.1127977862
0.460562725648
12 45.5031051423
0.368759971794
13 45.3963873277
0.488769149196
14 46.8574727474
0.398575818966
15 43.4129741227
0.435101048827
16 42.660342056
0.473070925268
17 46.3255272099
0.399645132203
18 47.9349976707
0.397802668002
19 48.7046082539
0.446494082546
20 46.4184291736
0.455057297432
21 45.8276170593
0.458748975446
22 45.5043539295
0.412746102763
23 47.3063313498
0.431180726291
24 45.3031680731
0.483234933199
25 46.4834806887
0.391886690737
26 46.1529531417
0.463826797345
27 44.51037182
0.446008506745
28 47.3377289377
0.363639333296
29 46.7828571429
0.426234884814
30 46.4706346285
0.501929024517
31 47.2458841971
0.40621943

# Local Depth

In [11]:
#local depth
filename = "data/" + area + "/" + "closeness_centrality.csv"

closeness_centrality = pd.read_csv(filename, index_col=0)
adjacent_node = adjacency_matrix.sum().to_frame()
adjacent_node.columns = ["closeness_centrality"]
adjacent_node.index = adjacent_node.index.map(int)
mean_depth = (closeness_centrality - adjacent_node) / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   

In [12]:
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_local_depth = - correlation_dataframe.corr().values[0][1]

In [13]:
correlation_local_depth

0.51097737493185613

# Connectivity 

In [14]:
mean_depth = adjacent_node / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_connectivity = correlation_dataframe.corr().values[0][1]
correlation_connectivity

0.54995175173751432

# Global Depth

In [15]:
mean_depth = closeness_centrality / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_global_depth = - correlation_dataframe.corr().values[0][1]
correlation_global_depth

0.51096262173349682